# GGR Trade Analysis - Staggered Portfolio Diagnostics

This notebook provides in-depth analysis of the GGR Distance Method using the **staggered portfolio methodology** from the original Gatev, Goetzmann, and Rouwenhorst (2006) paper.

## Staggered Approach
- New portfolio starts every ~21 trading days (monthly)
- Each portfolio: 12-month formation + 6-month trading
- At steady state: ~6 portfolios active simultaneously
- Monthly return = arithmetic average across active portfolios

## Analysis Sections

### 1. Staggered Performance Overview
- **Portfolio Timeline**: Gantt chart of overlapping portfolios
- **Returns Chart**: Cumulative returns + monthly returns + active count

### 2. Parameter Sensitivity (Robustness Check)
- **Heatmap**: Entry Threshold vs N Pairs, colored by Sharpe Ratio

### 3. Trade Diagnostics
- **MAE Analysis**: Maximum Adverse Excursion vs Final P&L
- **Duration Analysis**: Holding period histograms for Winners vs Losers

### 4. Exit Reason Analysis
- **Health Check Pie Chart**: Distribution of exit reasons
- **P&L by Exit Reason**: Violin plot of P&L by exit type

---

## 1. Setup & Configuration

In [ ]:
# Standard imports
import sys
import warnings
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import our modules
from src.data import fetch_or_load, get_close_prices, get_open_prices
from src.backtest import BacktestConfig
from src.staggered import StaggeredConfig, run_staggered_backtest
from src.analysis import (
    # Core metrics
    trades_to_dataframe,
    # Staggered-specific functions
    calculate_staggered_metrics,
    print_staggered_metrics,
    plot_staggered_returns,
    plot_portfolio_timeline,
    # Trade analysis functions
    plot_mae_analysis,
    plot_duration_histogram,
    plot_exit_reason_pie,
    plot_pnl_by_exit_reason,
)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("Modules loaded successfully!")

In [ ]:
# Configuration
CONFIG = {
    # Universe
    "symbols": ['DHT', 'FRO', 'ASC', 'ECO', 'NAT', 'TNK', 'INSW', 'TRMD', 'TOPS', 'TORO', 'PSHG'],
    
    # Date range
    "start_date": "2021-01-01",
    "end_date": "2026-01-01",
    
    # Staggered portfolio parameters (per GGR paper)
    "formation_days": 252,      # 12 months for pair selection + sigma
    "trading_days": 126,        # 6 months active trading window
    "overlap_days": 21,         # ~1 month between portfolio starts
    "n_pairs": 20,              # Top pairs selected per cycle
    
    # Trading parameters
    "entry_threshold": 2.0,
    "max_holding_days": 126,
    "capital_per_trade": 10000,
    "commission": 0.001,
}

# Parameter grid for sensitivity analysis
PARAM_GRID = {
    "entry_thresholds": [1.5, 2.0, 2.5, 3.0],
    "n_pairs_list": [10, 15, 20],
}

print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

print("\nParameter Grid:")
print(f"  Entry Thresholds: {PARAM_GRID['entry_thresholds']}")
print(f"  N Pairs: {PARAM_GRID['n_pairs_list']}")
print(f"  Total Combinations: {len(PARAM_GRID['entry_thresholds']) * len(PARAM_GRID['n_pairs_list'])}")

## 2. Data Loading

In [ ]:
# Fetch or load price data
prices = fetch_or_load(
    symbols=CONFIG["symbols"],
    start_date=CONFIG["start_date"],
    end_date=CONFIG["end_date"],
    cache_dir="data"
)

# Extract close and open prices
close_prices = get_close_prices(prices)
open_prices = get_open_prices(prices)

print(f"Data shape: {close_prices.shape}")
print(f"Date range: {close_prices.index[0].date()} to {close_prices.index[-1].date()}")
print(f"Trading days: {len(close_prices)}")
print(f"Symbols: {list(close_prices.columns)}")

In [ ]:
# Verify we have enough data for staggered approach
min_days_needed = CONFIG["formation_days"] + CONFIG["trading_days"]
print(f"Minimum days needed: {min_days_needed} (formation={CONFIG['formation_days']} + trading={CONFIG['trading_days']})")
print(f"Days available: {len(close_prices)}")

if len(close_prices) >= min_days_needed:
    print("Data sufficient for staggered backtest")
else:
    print("WARNING: Insufficient data for staggered backtest!")

## 3. Run Staggered Backtest

The staggered approach runs overlapping portfolio cycles per the GGR paper:
- Each cycle independently selects top pairs and runs a backtest
- Monthly returns are averaged across all active portfolios
- At steady state, 6 portfolios are active simultaneously

In [ ]:
# Create staggered configuration
staggered_config = StaggeredConfig(
    formation_days=CONFIG["formation_days"],
    trading_days=CONFIG["trading_days"],
    overlap_days=CONFIG["overlap_days"],
    n_pairs=CONFIG["n_pairs"],
    backtest_config=BacktestConfig(
        entry_threshold=CONFIG["entry_threshold"],
        max_holding_days=CONFIG["max_holding_days"],
        capital_per_trade=CONFIG["capital_per_trade"],
        commission=CONFIG["commission"],
    )
)

print("Staggered Configuration:")
print(f"  Formation: {staggered_config.formation_days} days")
print(f"  Trading: {staggered_config.trading_days} days")
print(f"  Overlap: {staggered_config.overlap_days} days")
print(f"  N Pairs: {staggered_config.n_pairs}")
print(f"  Entry Threshold: {staggered_config.backtest_config.entry_threshold}σ")

In [ ]:
# Run staggered backtest
print("Running staggered backtest...")

def progress_callback(current, total):
    if current % 5 == 0 or current == total:
        print(f"  Completed cycle {current}/{total}")

result = run_staggered_backtest(
    close_prices=close_prices,
    open_prices=open_prices,
    config=staggered_config,
    progress_callback=progress_callback,
)

# Get all trades and convert to DataFrame
all_trades = result.all_trades if result.all_trades else []
trades_df = trades_to_dataframe(all_trades)

# Add outcome column
if not trades_df.empty:
    trades_df['outcome'] = trades_df['pnl'].apply(lambda x: 'Winner' if x > 0 else 'Loser')

print(f"\nStaggered Backtest Complete!")
print(f"  Total portfolio cycles: {result.total_portfolios}")
print(f"  Total trades: {len(all_trades)}")
if all_trades:
    print(f"  Winners: {len([t for t in all_trades if t.pnl > 0])}")
    print(f"  Losers: {len([t for t in all_trades if t.pnl <= 0])}")

# Calculate and print staggered metrics
metrics = calculate_staggered_metrics(result)
print("\n")
print_staggered_metrics(metrics)

## 4. Staggered Portfolio Visualizations

### Portfolio Timeline
Shows how portfolios overlap over time. Each bar represents one portfolio cycle:
- **Blue section**: Formation period (pair selection + sigma calculation)
- **Green section**: Trading period (active trading)

At steady state, you should see ~6 overlapping trading periods.

In [ ]:
# Plot portfolio timeline (Gantt chart)
fig = plot_portfolio_timeline(result)
fig.show()

In [ ]:
# Plot staggered returns (3 panels: cumulative, monthly, active count)
fig = plot_staggered_returns(result)
fig.show()

## 5. Parameter Sensitivity Analysis (Robustness Check)

This heatmap shows how performance varies across different parameter combinations.

**What to look for:**
- A "stable plateau" of green (good Sharpe ratios) across multiple parameter values
- If changing parameters slightly destroys profits, the strategy may be overfitted

**Grid:**
- X-axis: Entry Threshold (1.5, 2.0, 2.5, 3.0 sigma)
- Y-axis: Number of Pairs (10, 15, 20)

In [ ]:
# Run staggered parameter grid search
print("Running parameter grid search (this may take several minutes)...\n")

grid_results = []
total_combos = len(PARAM_GRID["entry_thresholds"]) * len(PARAM_GRID["n_pairs_list"])
combo_num = 0

for n_pairs in PARAM_GRID["n_pairs_list"]:
    for entry_threshold in PARAM_GRID["entry_thresholds"]:
        combo_num += 1
        print(f"  [{combo_num}/{total_combos}] n_pairs={n_pairs}, entry_threshold={entry_threshold}σ")
        
        try:
            # Create config for this combination
            test_config = StaggeredConfig(
                formation_days=CONFIG["formation_days"],
                trading_days=CONFIG["trading_days"],
                overlap_days=CONFIG["overlap_days"],
                n_pairs=n_pairs,
                backtest_config=BacktestConfig(
                    entry_threshold=entry_threshold,
                    max_holding_days=CONFIG["max_holding_days"],
                    capital_per_trade=CONFIG["capital_per_trade"],
                    commission=CONFIG["commission"],
                )
            )
            
            # Run staggered backtest
            test_result = run_staggered_backtest(close_prices, open_prices, test_config)
            test_metrics = calculate_staggered_metrics(test_result)
            
            grid_results.append({
                "entry_threshold": entry_threshold,
                "n_pairs": n_pairs,
                "sharpe_ratio": test_metrics["sharpe_ratio"],
                "annualized_return": test_metrics["annualized_return"],
                "win_rate": test_metrics["win_rate"],
                "total_trades": test_metrics["total_trades"],
                "max_drawdown": test_metrics["max_drawdown"],
            })
        except Exception as e:
            print(f"    Error: {e}")
            grid_results.append({
                "entry_threshold": entry_threshold,
                "n_pairs": n_pairs,
                "sharpe_ratio": 0,
                "annualized_return": 0,
                "win_rate": 0,
                "total_trades": 0,
                "max_drawdown": 0,
            })

grid_df = pd.DataFrame(grid_results)
print("\nGrid search complete!")
grid_df

In [ ]:
# Plot Sharpe Ratio heatmap
pivot = grid_df.pivot(index="n_pairs", columns="entry_threshold", values="sharpe_ratio")

fig = go.Figure(data=go.Heatmap(
    z=pivot.values,
    x=[f"{x}σ" for x in pivot.columns],
    y=[f"{y} pairs" for y in pivot.index],
    colorscale="RdYlGn",
    text=[[f"{v:.2f}" for v in row] for row in pivot.values],
    texttemplate="%{text}",
    textfont={"size": 14},
    colorbar_title="Sharpe Ratio"
))

fig.update_layout(
    title="Parameter Sensitivity: Sharpe Ratio",
    xaxis_title="Entry Threshold",
    yaxis_title="Number of Pairs",
    height=400
)
fig.show()

In [ ]:
# Plot Win Rate heatmap
pivot = grid_df.pivot(index="n_pairs", columns="entry_threshold", values="win_rate")

fig = go.Figure(data=go.Heatmap(
    z=pivot.values,
    x=[f"{x}σ" for x in pivot.columns],
    y=[f"{y} pairs" for y in pivot.index],
    colorscale="RdYlGn",
    text=[[f"{v:.1%}" for v in row] for row in pivot.values],
    texttemplate="%{text}",
    textfont={"size": 14},
    colorbar_title="Win Rate"
))

fig.update_layout(
    title="Parameter Sensitivity: Win Rate",
    xaxis_title="Entry Threshold",
    yaxis_title="Number of Pairs",
    height=400
)
fig.show()

### Parameter Sensitivity Interpretation

**Healthy signs:**
- Adjacent cells have similar colors (stable performance)
- Multiple good configurations exist

**Warning signs:**
- Single bright green cell surrounded by red (overfitting)
- Extreme sensitivity to small parameter changes

## 6. Maximum Adverse Excursion (MAE) Analysis

**Hypothesis:** Losers aren't necessarily "wrong" pairs, but the entry timing was bad (entered too early on "falling knives").

**What to look for:**
- Cluster of losers at high MAE values (>4sigma) = entering too early
- Winners should have lower MAE (quick convergence)

**Action:** If losers cluster at high MAE, increase entry_threshold or add a volatility filter.

In [ ]:
# Plot MAE analysis
if all_trades:
    fig = plot_mae_analysis(all_trades)
    fig.show()
    
    # Summary statistics
    winners = [t for t in all_trades if t.pnl > 0]
    losers = [t for t in all_trades if t.pnl <= 0]
    
    if winners:
        winner_mae = [abs(t.max_adverse_spread) for t in winners]
        print(f"\nWinners MAE: Mean={np.mean(winner_mae):.2f}σ, Max={np.max(winner_mae):.2f}σ")
    
    if losers:
        loser_mae = [abs(t.max_adverse_spread) for t in losers]
        print(f"Losers MAE: Mean={np.mean(loser_mae):.2f}σ, Max={np.max(loser_mae):.2f}σ")
else:
    print("No trades to analyze")

## 7. Duration Analysis ("Stale Trade" Histogram)

**Hypothesis:** The longer a trade stays open, the lower the probability of profit (GGR implies good convergence happens quickly).

**What to look for:**
- **Winners**: Skewed left (fast profits, 5-20 days)
- **Losers**: Skewed right (long, slow bleeds)

**Action:** If losers are all >50 days, implement a Time Stop (e.g., close after 45 days).

In [ ]:
# Plot duration histogram
if all_trades:
    fig = plot_duration_histogram(all_trades)
    fig.show()
    
    # Summary
    winners = [t for t in all_trades if t.pnl > 0]
    losers = [t for t in all_trades if t.pnl <= 0]
    
    if winners:
        winner_days = [t.holding_days for t in winners]
        print(f"\nWinners Duration: Mean={np.mean(winner_days):.1f}d, Median={np.median(winner_days):.1f}d")
    
    if losers:
        loser_days = [t.holding_days for t in losers]
        print(f"Losers Duration: Mean={np.mean(loser_days):.1f}d, Median={np.median(loser_days):.1f}d")
else:
    print("No trades to analyze")

## 8. Exit Reason Analysis

### Health Check Pie Chart

**Question:** How often does the strategy actually work as intended?

**Interpretation:**
- **Healthy Strategy**: >70% "crossing" exits (pairs are mean-reverting)
- **Unhealthy Strategy**: >40% "max_holding" exits (pairs are drifting, tying up capital)

In [ ]:
# Plot exit reason pie chart
if all_trades:
    fig = plot_exit_reason_pie(all_trades)
    fig.show()
    
    # Print breakdown
    exit_counts = {}
    for t in all_trades:
        exit_counts[t.exit_reason] = exit_counts.get(t.exit_reason, 0) + 1
    
    print("\nExit Reason Breakdown:")
    for reason, count in sorted(exit_counts.items()):
        pct = count / len(all_trades) * 100
        print(f"  {reason}: {count} trades ({pct:.1f}%)")
else:
    print("No trades to analyze")

### P&L by Exit Reason ("Cost of Waiting")

**Question:** Are forced exits costing money, or just wasting time?

**What to look for:**
- "crossing" trades: Should have tight, positive P&L distribution
- "max_holding" trades: Will likely have wide variance (some huge losses, some small wins)

**Action:** If "max_holding" trades cause huge drawdowns, add a hard Stop Loss (e.g., at -10% P&L).

In [ ]:
# Plot P&L by exit reason
if all_trades:
    fig = plot_pnl_by_exit_reason(all_trades)
    fig.show()
    
    # Summary statistics by exit reason
    exit_pnl = {}
    for t in all_trades:
        if t.exit_reason not in exit_pnl:
            exit_pnl[t.exit_reason] = []
        exit_pnl[t.exit_reason].append(t.pnl)
    
    print("\nP&L Summary by Exit Reason:")
    for reason in sorted(exit_pnl.keys()):
        pnls = exit_pnl[reason]
        print(f"\n  {reason}:")
        print(f"    Count: {len(pnls)}")
        print(f"    Mean P&L: ${np.mean(pnls):,.2f}")
        print(f"    Win Rate: {len([p for p in pnls if p > 0]) / len(pnls):.1%}")
        print(f"    Total P&L: ${sum(pnls):,.2f}")
else:
    print("No trades to analyze")

## 9. Summary & Conclusions

In [ ]:
print("=" * 70)
print("GGR STAGGERED PORTFOLIO ANALYSIS SUMMARY")
print("=" * 70)

if all_trades:
    winners = [t for t in all_trades if t.pnl > 0]
    losers = [t for t in all_trades if t.pnl <= 0]
    
    print(f"\n1. STAGGERED BACKTEST RESULTS")
    print(f"   Total Portfolio Cycles: {result.total_portfolios}")
    print(f"   Peak Active Portfolios: {int(result.active_portfolios_over_time.max())}")
    print(f"   Total Trades: {len(all_trades)}")
    print(f"   Win Rate: {len(winners)/len(all_trades):.1%}")
    print(f"   Annualized Return: {metrics['annualized_return']:.1%}")
    print(f"   Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    
    print(f"\n2. PARAMETER SENSITIVITY")
    if len(grid_df) > 0:
        best_params = grid_df.loc[grid_df['sharpe_ratio'].idxmax()]
        print(f"   Best Configuration:")
        print(f"     Entry Threshold: {best_params['entry_threshold']}σ")
        print(f"     N Pairs: {int(best_params['n_pairs'])}")
        print(f"     Sharpe Ratio: {best_params['sharpe_ratio']:.2f}")
    
    print(f"\n3. DURATION ANALYSIS")
    if winners:
        print(f"   Winner Median Duration: {np.median([t.holding_days for t in winners]):.0f} days")
    if losers:
        print(f"   Loser Median Duration: {np.median([t.holding_days for t in losers]):.0f} days")
    
    print(f"\n4. EXIT ANALYSIS")
    crossing_exits = [t for t in all_trades if t.exit_reason == 'crossing']
    crossing_pct = len(crossing_exits) / len(all_trades) * 100
    print(f"   Convergence Rate: {crossing_pct:.1f}%")
    if crossing_pct >= 70:
        print(f"   Status: HEALTHY (>70% mean-reverting)")
    elif crossing_pct >= 50:
        print(f"   Status: MODERATE (50-70% mean-reverting)")
    else:
        print(f"   Status: NEEDS REVIEW (<50% mean-reverting)")
else:
    print("\nNo trades generated - check configuration and data.")

print("\n" + "=" * 70)

## Next Steps

Based on this analysis, consider:

1. **If high MAE on losers**: Increase entry threshold or add volatility filter
2. **If long duration on losers**: Implement a time-based stop (e.g., 45 days)
3. **If max_holding exits have large losses**: Add a P&L-based stop loss
4. **If parameter sensitivity shows instability**: Use more conservative parameters
5. **If convergence rate is low**: Consider tighter pair selection (lower n_pairs)